In [ ]:
import numpy as np
from numba import jit

In [ ]:
N = 20
L = 10

Definir funciones para el programa:

Notación:
- L, indica la longitud de la caja
- T, tiempo total a calcular
- r, vector con posiciones de las partículas. Tiene dimensiones (N, 2), con N el número de partículas
- R, matriz con la distancia entre partículas

In [ ]:
@jit(nopython = True)
def cond_contorno(r):
    return r%L

@jit(nopython = True)
def lennard_jones(r):
    R = compute_distance(r)
    


    return 0

def compute_distance(r):
    R = np.zeros((N, N, 2))
    for i in range(1, N):
        for j in range(i, N):
            R[i, j] = r[j]- r[i]
            angle = np.arctan(R[i, j, 1]/R[i, j, 0])
            max_length = abs(L/np.cos(angle%(np.pi/4)))
            norm = np.linalg.norm(R[i, j])
            if(norm > max_length):
                R[i, j] =  -R[i, j] *(max_length - norm) / norm
            R[j, i] = R[i, j]
    return R
